# Telecom Churn
- Team 1
- Team members
            Tashi Chotso - 20BDA01
            Jerry Raju Mathew - 20BDA16
            Gunam Ramya Sri - 20BDA33
            PrajjWal Patel - 20BDA48

## Problem statement
- In the telecom industry, customers are able to choose from multiple service providers and actively switch from one operator to another. In this highly competitive market, the telecommunications industry experiences an average of 15-25% annual churn rate. Given the fact that it costs 5-10 times more to acquire a new customer than to retain an existing one, customer retention has now become even more important than customer acquisition or reduce customer churn, telecom companies need to predict which customers are at high risk of churn.

1. Extract, load, and read the data as text files as RDD Transform    
2. Transform: Exploratory data analysis using rdd

    - A. Replace Contract column values
        - Month-to-month -1m
        - One Year - 1y
        - Two Year - 2y
        - rest all - Others
        
    - B. Unique customer count

    - C. Describe the categorical and numerical columns seperately

    - D. GroupBy contract and avg of totalcharges

    - E. Using accumulator add the totalcharges                         
    
         
3. Load: Save analysis report

    - GroupBy contract and avg of totalcharges save as files

In [81]:
import pyspark
import os
from random import random
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession
import numpy as np
from pyspark.mllib.stat import Statistics

The first thing a Spark program must do is to create a SparkContext object, which tells Spark how to access a cluster. To create a SparkContext you first need to build a SparkConf object that contains information about your application.

## 1. Extract, load, and read the data as text files as RDD Transform

In [82]:
spark = SparkSession.builder.master("local").\
        appName("SparkApplication").\
        config("spark.driver.bindAddress","localhost").\
        config("spark.ui.port","4041").\
        getOrCreate()

In [83]:
spark

### To read CSV  in Spark into single RDD.

In [84]:
sc = spark.sparkContext

In [85]:
#2.read csv file in rdd
data=sc.textFile("telecomChurn.csv")

In [86]:
print('\n',data) #what file rdd


 telecomChurn.csv MapPartitionsRDD[287] at textFile at NativeMethodAccessorImpl.java:0


In [87]:
print('\n',type(data)) # variable type


 <class 'pyspark.rdd.RDD'>


In [88]:
print('\n',dir(data))# what attributes are avaiable 



 ['__add__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_computeFractionForSampleSize', '_defaultReducePartitions', '_id', '_is_barrier', '_jrdd', '_jrdd_deserializer', '_memory_limit', '_pickled', '_reserialize', '_to_java_object_rdd', 'aggregate', 'aggregateByKey', 'barrier', 'cache', 'cartesian', 'checkpoint', 'coalesce', 'cogroup', 'collect', 'collectAsMap', 'collectWithJobGroup', 'combineByKey', 'context', 'count', 'countApprox', 'countApproxDistinct', 'countByKey', 'countByValue', 'ctx', 'distinct', 'filter', 'first', 'flatMap', 'flatMapValues', 'fold', 'foldByKey', 'foreach', 'foreachPartition', 'fullOuterJoin', 'getCheckpointFile', 'getNumPartitions', 'getResourc

In [89]:
#header
header=data.first()
print(header)

customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn


In [90]:
# remove header
rdd1= data.filter(lambda line: line !=header)

In [91]:
#total record counts
print('\n file has:',rdd1.count(),'row') #counts



 file has: 7043 row


In [92]:
#filter first row
print('\n file line:',rdd1.first())


 file line: 7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No


In [93]:
rdd1.take(5) #take file element

['7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No',
 '5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No',
 '3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes',
 '7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.3,1840.75,No',
 '9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.7,151.65,Yes']

In [94]:
#total unique records count
rdd1.distinct().count()

7043

In [95]:
print("initial partition count:"+str(rdd1.getNumPartitions()))
#Outputs: initial partition count:2

initial partition count:1


In [96]:
step1= rdd1.map(lambda line: line.split(",")) # split by ,

In [97]:
step1.take(2)

[['7590-VHVEG',
  'Female',
  '0',
  'Yes',
  'No',
  '1',
  'No',
  'No phone service',
  'DSL',
  'No',
  'Yes',
  'No',
  'No',
  'No',
  'No',
  'Month-to-month',
  'Yes',
  'Electronic check',
  '29.85',
  '29.85',
  'No'],
 ['5575-GNVDE',
  'Male',
  '0',
  'No',
  'No',
  '34',
  'Yes',
  'No',
  'DSL',
  'Yes',
  'No',
  'Yes',
  'No',
  'No',
  'No',
  'One year',
  'No',
  'Mailed check',
  '56.95',
  '1889.5',
  'No']]

## 2. Transform: Exploratory data analysis using rdd

##   A. Replace Contract column values

    - Month-to-month -1m
    - One Year - 1y
    - Two Year - 2y
    - rest all - Others

In [98]:
#replaced column value
def replace(column_val):
    if column_val=="Month-to-month":
        column_val="1m"
    elif column_val=="One year":
        column_val="1y"
    elif column_val=="Two year":
        column_val="2y"
    else:
        column_val= "Others"
    return column_val

In [99]:
# Contract column values passed to the replace function and assigned new column value
step2=step1.map(lambda x: (x[0],x[1],x[2],x[3],x[4],x[5],
                           x[6],x[7],x[8],x[9],x[10],x[11],x[12],x[13],x[14],replace(x[15]),x[16],x[17],x[18],x[19],x[20]))

In [100]:
#show first two records 
step2.take(2)

[('7590-VHVEG',
  'Female',
  '0',
  'Yes',
  'No',
  '1',
  'No',
  'No phone service',
  'DSL',
  'No',
  'Yes',
  'No',
  'No',
  'No',
  'No',
  '1m',
  'Yes',
  'Electronic check',
  '29.85',
  '29.85',
  'No'),
 ('5575-GNVDE',
  'Male',
  '0',
  'No',
  'No',
  '34',
  'Yes',
  'No',
  'DSL',
  'Yes',
  'No',
  'Yes',
  'No',
  'No',
  'No',
  '1y',
  'No',
  'Mailed check',
  '56.95',
  '1889.5',
  'No')]

In [101]:
## function to convert numerical columns from string to int
def string_to_int(val):
    try:
        return int(float(val))
    except:
        return 0

In [102]:
#Column 2,5,18,19 are numberical column so apply string_to_int function for converting string to int
step3=step2.map(lambda x: (x[0],x[1],string_to_int(x[2]),x[3],x[4],string_to_int(x[5]),
                           x[6],x[7],x[8],x[9],x[10],x[11],x[12],x[13],x[14],x[15],x[16],x[17],string_to_int(x[18]),string_to_int(x[19]),x[20]))

In [103]:
# show first record
step3.take(1)

[('7590-VHVEG',
  'Female',
  0,
  'Yes',
  'No',
  1,
  'No',
  'No phone service',
  'DSL',
  'No',
  'Yes',
  'No',
  'No',
  'No',
  'No',
  '1m',
  'Yes',
  'Electronic check',
  29,
  29,
  'No')]

## C. Describe the categorical and numerical columns seperately

### i. Numerical data

In [104]:
#show only numerical columns in the given dataset
num_data=step3.map(lambda x: (x[2],x[5],x[18],x[19]))

In [105]:
#show first four records
num_data.take(4)

[(0, 1, 29, 29), (0, 34, 56, 1889), (0, 2, 53, 108), (0, 45, 42, 1840)]

#### mean and variance of numerical data

In [145]:
#import necessary packages 
import numpy as np
from pyspark.mllib.stat import Statistics
num_data=step3.map(lambda x: (x[2],x[5],x[18],x[19])) # numberical cols 
num_data.take(5)
summary = Statistics.colStats(num_data)
print("------ statistic analysis -----------")
print("mean :",summary.mean())  # a dense vector containing the mean value for each column
print("variance :",summary.variance())  # column-wise variance


------ statistic analysis -----------
mean : [1.62146812e-01 3.23711487e+01 6.42958966e+01 2.27926509e+03]
variance : [1.35874516e-01 6.03168108e+02 9.05572188e+02 5.13834082e+06]


#### Maximum and minimum entry of numerical data

In [146]:
#aggregate function for numerical columns
print("------statistic analysis----------------------")
print(" Maximum  entry  in each column",num_data.max())
print("Minimum  entery  in each column",num_data.min())

------statistic analysis----------------------
 Maximum  entry  in each column (1, 72, 117, 8436)
Minimum  entery  in each column (0, 0, 19, 0)


### ii. Categorical data

In [108]:
#show categorical columns 
cat_data=step3.map(lambda x:(x[0],x[1],x[3],x[4],x[6],[7],x[8],x[9],x[10],x[11],x[12],x[13],x[14],x[15],x[16],x[17],x[20]))
#show first records of category data
cat_data.take(1)

[('7590-VHVEG',
  'Female',
  'Yes',
  'No',
  'No',
  [7],
  'DSL',
  'No',
  'Yes',
  'No',
  'No',
  'No',
  'No',
  '1m',
  'Yes',
  'Electronic check',
  'No')]

In [109]:
#Number of records 
cat_data.count() 

7043

In [110]:
##User defined functions 
# Module for changing string to int
def string_to_int(val):
    try:
        return int(float(val))
    except:
        return 0

# Module getting the count
def count(x):
    temp=0
    for val in list(x):
        temp = temp + val[1]
        out = str(val[0])+","+str(temp)
    return (out)

#module for average
def mean_val(x):
    sums=0
    l=0
    for i in x:
        sums= sums + i[1]
        l=l+1
        avg=round(sums/l,2)
        
    return (avg)

## module for replace value
def replace(column_val):
    if column_val=="Month-to-month":
        column_val="1m"
    elif column_val=="One year":
        column_val="1y"
    elif column_val=="Two year":
        column_val="2y"
    else:
        column_val= "Others"
    return column_val


### count Churn
- Counting number of customer  churn "Yes"  or not churn "No"

In [144]:
# excluding header, split data by ',', count number of customer churn "Yes" or not churn "No"
col=[0,1,3,4,6,7,8,9,10,11,12,13,14,15,16,17,20]
cat_count = data.filter(lambda line: line != header).\
            map(lambda line: line.split(",")).\
            map(lambda x: (x[0],x[1],x[2],x[3],x[4],x[5],
                           x[6],x[7],x[8],x[9],x[10],x[11],x[12],x[13],x[14],replace(x[15]),x[16],x[17],x[18],x[19],x[20])).map(
    lambda x: ((x[20]), string_to_int(x[5]))).\
    groupBy(lambda x: x[0]).\
    map(lambda x: (count(x[1]))).coalesce(1)
print("----count number of customer churn or not churn ------------")
cat_count.collect()

----count number of customer churn or not churn ------------


['No,194387', 'Yes,33603']

In [143]:
#Count number of gender "Female" and 'Male'
cat_ge = data.filter(lambda line: line != header).\
            map(lambda line: line.split(",")).\
           map(
    lambda x: ((x[1]), string_to_int(x[5]))).\
    groupBy(lambda x: x[0]).\
    map(lambda x: (count(x[1]))).coalesce(1)
print("----------Count number of famale and male-------- ")
print(cat_ge.collect())

----------Count number of famale and male-------- 
['Female,112469', 'Male,115521']


### Distribution of different internet plans 

In [141]:
#Count number of mobile services such as DSL,Fiber and No
cat_service = data.filter(lambda line: line != header).\
            map(lambda line: line.split(",")).map(
    lambda x: ((x[8]), string_to_int(x[5]))).\
    groupBy(lambda x: x[0]).\
    map(lambda x: (count(x[1]))).coalesce(1)
print("-------shown different internet plans----------")
print(cat_service.collect())

-------shown different internet plans----------
['DSL,79461', 'Fiber optic,101914', 'No,46615']


###  Distribution of different PaymentMethod

In [139]:
#counting number of payment method such as electronic check, mailed check, bang transfer, and credit card
cat_pyMethod = data.filter(lambda line: line != header).\
            map(lambda line: line.split(",")).map(
    lambda x: ((x[17]), string_to_int(x[5]))).\
    groupBy(lambda x: x[0]).\
    map(lambda x: (count(x[1]))).coalesce(1)
print("-----shown different Payment Method-----------")
print(cat_pyMethod.collect())

-----shown different Payment Method-----------
['Electronic check,59538', 'Mailed check,35190', 'Bank transfer (automatic),67406', 'Credit card (automatic),65856']


### Distribution of different service Lines

In [137]:
#counting number of serivice lines such as "no phone service",'No','Yes'
cat_pyMethod = data.filter(lambda line: line != header).\
            map(lambda line: line.split(",")).map(
    lambda x: ((x[7]), string_to_int(x[5]))).\
    groupBy(lambda x: x[0]).\
    map(lambda x: (count(x[1]))).coalesce(1)
print("----------Different service Lines-------- ")
cat_pyMethod.collect()

----------Different service Lines-------- 


['No phone service,21645', 'No,81817', 'Yes,124528']

### Number of  customer churn w.r.t gender

In [131]:
# couting number of churn or not churn with respect to the gender
cat_gender = data.filter(lambda line: line != header).\
            map(lambda line: line.split(",")).map(
    lambda x: ((x[20],x[1]), string_to_int(x[5]))).\
    groupBy(lambda x: x[0]).\
    map(lambda x: (count(x[1]))).coalesce(1)

print("---Number of customer churn and not churn w.r.t gender----")
print(cat_gender.collect())

---Number of customer churn and not churn w.r.t gender----
["('No', 'Female'),96502", "('No', 'Male'),97885", "('Yes', 'Male'),17636", "('Yes', 'Female'),15967"]


## B. Unique customer count

In [133]:
cat_customer = data.filter(lambda line: line != header).\
            map(lambda line: line.split(",")).map(
    lambda x: ((x[0]), string_to_int(x[5]))).\
    groupBy(lambda x: x[0]).\
    map(lambda x: (count(x[1]))).coalesce(1)

print("Unique Customer")
print("show first three unique customer",cat_customer.take(3))
print("----------------------------------------------------------------------")
print("Number of unique customer",cat_customer.distinct().count())

Unique Customer
show first three unique customer ['7590-VHVEG,1', '5575-GNVDE,34', '3668-QPYBK,2']
----------------------------------------------------------------------
Number of unique customer 7043


## D.  GroupBy contract and avg of totalcharges 

In [76]:
#show contract and totalcharge column
step4=step3.map(lambda x: (x[15],x[19]))
step4.take(2)

[('1m', 29), ('1y', 1889)]

In [134]:
 #groupby contract and avg totalcharges   
def mean_val(x):
    sums=0
    l=0
    for i in x:
        sums= sums + i[1]
        l=l+1
        avg=round(sums/l,2)
        
    return (avg)
    
step5= step4.map(lambda x:((x[0]),string_to_int(x[1])))\
            .map(lambda x: (x[0],x[1]))\
            .groupBy(lambda x: (x[0])).\
            map(lambda x: (x[0],mean_val(x[1]))).coalesce(1)

print("---------Average totalcharges in each of 1m,1y and 2y--------------")           
step5.collect() 

---------Average totalcharges in each of 1m,1y and 2y--------------


[('1m', 1368.79), ('1y', 3032.14), ('2y', 3706.47)]

#### As we can see above analysis,  average total charge in 1m is ->1368.79, 1y->3032.14 and 2y->3706.47

## E. Using accumulator add the totalcharges

In [136]:
#starting from 0 and add up all the totalcharges 
accuSum=spark.sparkContext.accumulator(0)
def countFun(x):
    global accuSum
    accuSum+=x
totalcharges.foreach(countFun)
print("-----Total Charges-------------------")
print("Total Charges occured: ", accuSum.value)
print("---------------------------------------")

accumCount=spark.sparkContext.accumulator(0)
totalcharges.foreach(lambda x:accumCount.add(1))
print("Number of customer entries: ", accumCount.value)

-----Total Charges-------------------
Total Charges occured:  16052864
---------------------------------------
Number of customer entries:  7043


# 3. Load: Save analysis report

- GroupBy contract and avg of totalcharges save as files

In [42]:
# save textfile in folder Report with file name avg_totalcharges
step5.saveAsTextFile("Report/avg_totalCharges")

--------------------------------------------------------END-------------------------------------------------------------------